A first attempt at getting a working LGBM Model running

It works, but the scores are terrible

The best model is pickled to 'model/best_LGBM.pkl'

# IMPORTS

In [1]:
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score, roc_curve
import xgboost as xgb
from xgboost.sklearn import XGBRegressor
import numpy as np
from lightgbm import LGBMClassifier
import re
import gc

# READ DATA

In [50]:
USE_SAVED_DATA = True

if USE_SAVED_DATA:
    print ('Using saved data')
    df = pd.read_csv('testdata/hdcr10k.csv')
else:
    print ('Using FULL data')
    df = pd.read_csv('../data/cleaned/hcdr_FULL.csv',nrows=10000)
    df = df.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
    df.to_csv('testdata/hdcr10k.csv', index=False)
df.shape

Using saved data


(10000, 798)

In [51]:
df.TARGET.value_counts()

0.0    9225
1.0     775
Name: TARGET, dtype: int64

In [52]:
print (df.isna().sum().sum())


2058717


# IMPUTE NaN

In [53]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.impute import SimpleImputer

df.replace([np.inf, -np.inf], np.nan, inplace=True)

imputer = SimpleImputer(missing_values=np.nan, strategy = 'median')
df=pd.DataFrame(imputer.fit_transform(df), columns=df.columns, index=df.index)

# Scale the features
scaler = MinMaxScaler(feature_range = (0, 1))
df=pd.DataFrame(scaler.fit_transform(df), columns=df.columns, index=df.index)


In [44]:
df = df[[
    'EXT_SOURCE_3',
    'EXT_SOURCE_2',
    'PAYMENT_RATE',
    'DAYS_ID_PUBLISH',
    'EXT_SOURCE_1',
    'DAYS_BIRTH',
    'DAYS_EMPLOYED',
    'DAYS_EMPLOYED_PERC',
    'REGION_POPULATION_RELATIVE',
    'DAYS_REGISTRATION',
    'ANNUITY_INCOME_PERC',
    'DAYS_LAST_PHONE_CHANGE',
    'AMT_GOODS_PRICE',
    'AMT_ANNUITY',
    'INCOME_CREDIT_PERC',
    'AMT_CREDIT',
    'INCOME_PER_PERSON',
    'AMT_INCOME_TOTAL',
    'HOUR_APPR_PROCESS_START',
    'TARGET']]


In [45]:
print (df.isna().sum().sum())

0


# X and y

In [46]:
X= df.drop('TARGET',axis=1)
y= df['TARGET']

print (X.shape)
print (y.shape)

(10000, 19)
(10000,)


# SPLIT TEST/TRAIN

In [47]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=1234)

# GRIDSEARCHCV

In [60]:
from sklearn.model_selection import GridSearchCV

folds = KFold(n_splits= 5, shuffle=True, random_state=1001)

params = {
    'n_estimators': [100],
    'learning_rate':[0.1,0.02],
    'num_leaves':[34,40],
    'colsample_bytree':[0.9497036],
    'subsample':[0.8715623],
    'max_depth':[5, 8],
    'reg_alpha':[0.041545473],
    'reg_lambda':[0.0735294],
    'min_split_gain':[0.0222415],
    'min_child_weight':[39.3259775],
}

grid = GridSearchCV(LGBMClassifier(random_state=0), params, scoring='roc_auc', cv=folds, verbose=3)
grid.fit(X_train, y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV 1/5] END colsample_bytree=0.9497036, learning_rate=0.1, max_depth=5, min_child_weight=39.3259775, min_split_gain=0.0222415, n_estimators=100, num_leaves=34, reg_alpha=0.041545473, reg_lambda=0.0735294, subsample=0.8715623;, score=0.749 total time=   0.0s
[CV 2/5] END colsample_bytree=0.9497036, learning_rate=0.1, max_depth=5, min_child_weight=39.3259775, min_split_gain=0.0222415, n_estimators=100, num_leaves=34, reg_alpha=0.041545473, reg_lambda=0.0735294, subsample=0.8715623;, score=0.761 total time=   0.0s
[CV 3/5] END colsample_bytree=0.9497036, learning_rate=0.1, max_depth=5, min_child_weight=39.3259775, min_split_gain=0.0222415, n_estimators=100, num_leaves=34, reg_alpha=0.041545473, reg_lambda=0.0735294, subsample=0.8715623;, score=0.712 total time=   0.0s
[CV 4/5] END colsample_bytree=0.9497036, learning_rate=0.1, max_depth=5, min_child_weight=39.3259775, min_split_gain=0.0222415, n_estimators=100, num_leaves=34, re

GridSearchCV(cv=KFold(n_splits=5, random_state=1001, shuffle=True),
             estimator=LGBMClassifier(random_state=0),
             param_grid={'colsample_bytree': [0.9497036],
                         'learning_rate': [0.1, 0.02], 'max_depth': [5, 8],
                         'min_child_weight': [39.3259775],
                         'min_split_gain': [0.0222415], 'n_estimators': [100],
                         'num_leaves': [34, 40], 'reg_alpha': [0.041545473],
                         'reg_lambda': [0.0735294], 'subsample': [0.8715623]},
             scoring='roc_auc', verbose=3)

# RESULTS

In [61]:
print (grid.best_score_)
print(grid.best_estimator_)
print(grid.best_params_)

0.7437966330993124
LGBMClassifier(colsample_bytree=0.9497036, max_depth=5,
               min_child_weight=39.3259775, min_split_gain=0.0222415,
               num_leaves=34, random_state=0, reg_alpha=0.041545473,
               reg_lambda=0.0735294, subsample=0.8715623)
{'colsample_bytree': 0.9497036, 'learning_rate': 0.1, 'max_depth': 5, 'min_child_weight': 39.3259775, 'min_split_gain': 0.0222415, 'n_estimators': 100, 'num_leaves': 34, 'reg_alpha': 0.041545473, 'reg_lambda': 0.0735294, 'subsample': 0.8715623}


# EXTRACT BEST MODEL

In [62]:
from sklearn.metrics import roc_auc_score

best_model = grid.best_estimator_

best_model.fit(X_train, y_train)
y_predict = best_model.predict(X_test)
print ("ROC AUC score:",roc_auc_score(y_test,y_predict))

ROC AUC score: 0.5080261136712749


# SAVE MODEL

In [63]:
import joblib

joblib.dump(best_model, 'model/best_LGBM.pkl')

['model/best_LGBM.pkl']

# XGBOOST

In [64]:
xgb_model = xgb.XGBClassifier()

In [65]:
from sklearn.model_selection import GridSearchCV

folds = KFold(n_splits= 5, shuffle=True, random_state=1001)

parameters = {'nthread':[4], #when use hyperthread, xgboost may become slower
              'objective':['binary:logistic'],
              'learning_rate': [0.05], #so called `eta` value
              'max_depth': [6],
              'min_child_weight': [11],
              'silent': [1],
              'subsample': [0.8],
              'colsample_bytree': [0.7],
              'n_estimators': [5], #number of trees, change it to 1000 for better results
              'missing':[-999],
              'seed': [1337]}


grid = GridSearchCV(xgb_model, params, scoring='roc_auc', cv=folds, verbose=3, refit=True)
grid.fit(X_train, y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[10:58:18] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "min_split_gain", "num_leaves" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[CV 1/5] END colsample_bytree=0.9497036, learning_rate=0.1, max_depth=5, min_child_weight=39.3259775, min_split_gain=0.0222415, n_estimators=100, num_leaves=34, reg_alpha=0.041545473, reg_lambda=0.0735294, subsample=0.8715623;, score=0.751 total time=   0.4s
[10:58:18] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "min_split_gain", "num_leaves" } might not be used.

  This could be a false alarm, with some parameters getting used by languag

GridSearchCV(cv=KFold(n_splits=5, random_state=1001, shuffle=True),
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     gamma=None, gpu_id=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constrai...
                                     num_parallel_tree=None, predictor=None,
                                     random_state=None, reg_alpha=None,
                                     reg_lambda=None, ...),
             param_grid={'colsample_bytree': [0.9497036],
                         'learning_rate': [0.1, 0.02],

In [66]:
print (grid.best_score_)
print(grid.best_estimator_)
print(grid.best_params_)

0.7448052403819625
XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1,
              colsample_bytree=0.9497036, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, gamma=0, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.1, max_bin=256,
              max_cat_to_onehot=4, max_delta_step=0, max_depth=5, max_leaves=0,
              min_child_weight=39.3259775, min_split_gain=0.0222415,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_leaves=34, num_parallel_tree=1, predictor='auto',
              random_state=0, ...)
{'colsample_bytree': 0.9497036, 'learning_rate': 0.1, 'max_depth': 5, 'min_child_weight': 39.3259775, 'min_split_gain': 0.0222415, 'n_estimators': 100, 'num_leaves': 34, 'reg_alpha': 0.041545473, 'reg_lambda': 0.0735294, 'subsample': 0.8715623}


In [67]:
from sklearn.metrics import roc_auc_score

best_model = grid.best_estimator_

best_model.fit(X_train, y_train)
y_predict = best_model.predict(X_test)
print ("ROC AUC score:",roc_auc_score(y_test,y_predict))

[10:58:51] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "min_split_gain", "num_leaves" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


ROC AUC score: 0.5080261136712749


In [68]:
import joblib

joblib.dump(best_model, 'model/best_XGBOOST.pkl')

['model/best_XGBOOST.pkl']